In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')
import json

In [2]:
from time import time
from src.utils.CacheAdapter import FlatAdapter, EXP_END_OF_DATA, createAdapter_02_04_25_GOOD

In [3]:
adapter = FlatAdapter("train_02-04-25")
#print(adapter.load(1))
#print(adapter.load(3))
#print(adapter.load(3))
#adapter.save([{"a" : [1,2,3,7,8], "b" : [4]}, {"a" : [9,5, 8], "b" : [6]}])

[{'a': [1, 2, 3, 7, 8], 'b': [4]}, {'a': [9, 5, 8], 'b': [6]}]

In [4]:
adapter = FlatAdapter("train_02-04-25")
print(adapter.load(2))

Exception ignored in: <function FlatAdapter.__del__ at 0x7fd9a03bfd80>
Traceback (most recent call last):
  File "/home/trukhinmaksim/src/src/utils/CacheAdapter.py", line 33, in __del__
    super().__del__()
    ^^^^^^^^^^^^^^^
AttributeError: 'super' object has no attribute '__del__'


[{'a': [1, 2, 3, 4, 5, 6, 7, 8], 'b': [4]}]
